# Testing multielectrode GEMsort algorithm by using 16-channel synthesized data

In [ ]:
from numpy import apply_along_axis as apply
from scipy.signal import fftconvolve
from scipy import signal
from GEMsort_func import GEMsort
from Smoothing_data import Smoothing_filter
from Cut_func import cut
from scipy.signal import butter, lfilter
from copy import deepcopy
from numpy.linalg import svd
from numpy.linalg import svd
from mpl_toolkits.mplot3d import Axes3D
import copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import struct
import time
import numpy.matlib
import numpy.matlib


## Choose one of the datasets in the below cell (generated by the data file in this package)

In [ ]:
#loading and plotting the data
np.random.seed(10)
path = '../../data/Multi_16channels/'

# data=np.load(path + '/multi_16ch_usualdata.npy', allow_pickle=True)
data = np.load(path + '/multi_16ch_samecells.npy', allow_pickle=True) #this is the similar cells (for two cells)  dataset
# data=np.load(path + '/multi_16ch_sametime.npy', allow_pickle=True) #this is the same firing time (for two cells) dataset

X = data

%matplotlib inline
for i in range(16):
    plt.plot(X[i,:][0:10000])
plt.show()

### If you are running the data for same time firing cells or similar cells of channels i, j (i.e., multi_16ch_sametime or multi_16ch_samecells), uncomment below cell to see the same time effect

In [ ]:
############ for multi_16ch_sametime data or multi_16ch_samecells ############
# i = 0
# j = 3
# # Size = 10000 #if data is multi_16ch_sametime 
# Size = 1000 #if data is multi_16ch_samecells   


# plt.plot(X[i,:][0:Size], c='r')
# plt.plot(X[j,:][0:Size], c= 'b')
# plt.show()

In [ ]:
#Smoothing filter
#%matplotlib qt

for i in range(16):
    data_raw = X[i, :]
    plt.plot(data_raw)
    filtered_data = Smoothing_filter(data_raw, low=10, high=5000, sf=80000, order=2)
    plt.plot(filtered_data, c='r')
#     plt.show()
    
    X[i, :] = filtered_data

# plt.plot(data_raw[1:L,:])
# plt.plot(filtered_data[1:L,:],c='r')
# plt.show()

#short view
# for i in range(16):
#     plt.plot(X[i, :])
# plt.xlim([0, 10000])
# plt.ylim([-0.3, 0.3])
# plt.show()

# %matplotlib qt

# for i in range(16):
#     y = np.zeros((1, 1000)) + i + 5
#     plt.plot(X[i, :][0: 1000] + i) # +i is only for plotting figures with verrtical distace in one fig
#     plt.show()

In [ ]:
# plot heatmap 
np.random.seed(0)
sns.set_theme()
uniform_data = X[:, 0:2000]
# ax = sns.heatmap(uniform_data, cmap="YlGnBu", xticklabels=False)#, vmin=0.2, vmax=1) #cmap="Greens","BuPu","Blues"
# ax = sns.heatmap(uniform_data, cmap="YlGnBu", xticklabels=False, center=0)#, vmin=0.2, vmax=1)

uniform_data = X[:, 0:1000]
ax = sns.heatmap(uniform_data, cmap="YlGnBu", xticklabels=False)#, vmin=0.2, vmax=1)
# ax = sns.heatmap(uniform_data, cmap="YlGnBu", xticklabels=False, center=0)#, vmin=0.2, vmax=1)

In [ ]:
##### main program for getting spikes of channels #####
start_time = time.time()

#stage 1: detecting peaks of all channels
T1 = 64
ch_num = 16
T = X.shape[1] #length of data
N = T
Tdis = 50

p_all = np.array([], dtype = object).reshape((1,0))
cut_ch = []

for k in range (ch_num): 
    cut_ch .append([])

for i in range (16): #for every channel
    #print(i)
    x = X[i,:] 
    sigma = np.median(np.abs(x) / 0.6745)
    Tp = 4 * sigma  
    p,_ = scipy.signal.find_peaks(x, height = Tp, distance = Tdis) #np.abs(x) 
    
    if i == 15:
        p,_ = scipy.signal.find_peaks(np.abs(x), height = Tp, distance = Tdis)
        
    for j in range(np.array(p).shape[0]):
         cut_ch[i].append(cut(x,p[j],T1)[0]) 
    
    if p == []:
        p = [0] #to prevent error, so in every row if p=0 means: no peak in that channel
    y_arr = np.array([], dtype = np.int32)
    y = p
    y_arr = np.append(y_arr,y)
    #print(y_arr)
    p_all = np.append(p_all, 0)
    #print(p_all)
    p_all[-1] = y_arr.astype(int) 
 
print("--- %s seconds ---" % (time.time() - start_time))


# %matplotlib inline
# for i in range(np.array(cut_ch).shape[1]):
#     plt.plot(np.array(cut_ch)[0, i, :])
# plt.show()       

# for i in range(np.array(cut_ch).shape[1]):
#     plt.plot(np.array(cut_ch)[0, i, :])
#     plt.show() 

In [ ]:
#raster plot for cells' spikes
%matplotlib qt
path = '/.../results'
colors = ['dodgerblue', 'skyblue', 'gray','deeppink','orange','saddlebrown','lawngreen', 'g','black','cyan','red','brown','lightgray','yellow','pink','magenta']
main_cells_channels = [0, 3, 5, 7, 9, 11, 13, 15]
cells = range(1, 9)
length_signal = 20000 

fig = plt.figure(figsize=(10, 10), dpi=300)
ax = fig.add_subplot(1, 1, 1) 
ax.set_facecolor('xkcd:white')

print('p_all[i][0:length_signal].shape = ', p_all[i][0:length_signal].shape)
print('x.shape=', x.shape)
xticks = [0, 50, 100, 150, 200, 250, 300, 350, 400]
k = 0
for i in main_cells_channels:
    k = k+1
    x = np.zeros((1, p_all[i].shape[0])) + k
    ax.scatter(p_all[i], x, color=colors[i], marker='|', linewidth=0.6)
    ax.set(xticklabels=xticks)

plt.title('Spike raster plot', fontsize=10)
plt.ylabel('Neuron', fontsize=10)
plt.yticks( cells, fontsize=10)
plt.xlabel('Time (msec)', fontsize=10)
plt.xticks(fontsize=10)
plt.xlim(0, length_signal)



In [ ]:
start_time = time.time()

a = np.hstack(p_all) #put all 16 channels in a row for comparing all times 
rowp = a[None, :]

channels = []
times = []

#finding the similar peaks in near times and channels
for k in range (np.array(rowp).shape[1]): 
    channels.append([]) 
    times.append([])
        
for k in range (np.array(rowp).shape[1]): #for every peak  
    for i in range (p_all.shape[0]): #number of channels 
        for j in range (p_all[i].shape[0]):
            if all ([p_all[i][j] != 0 and np.abs(p_all[i][j] - rowp[0,k]) < 2 and X[i,rowp[0,k]] != 0]): 
#                 print(p_all[i][j])
#                 print(rowp[0,k])
                channels[k].append(i) #all the channels number with the same peak(k)
                times[k].append(j) #time values for that channel
                
# rowtt = np.unique(rowf)
# for i in range(rowtt.shape[0]):
#     if np.array(rowtt[i]).shape[0] == 1:
#         print(i)
#         channels[1,i] = -1
        
#old_sig = np.copy(signals) 
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#calculate correlations of all spikes happening in near times  
#Determining the channel with maximum correlation for the next part
%matplotlib inline

start_time = time.time()
signals_after_corr = deepcopy(cut_ch) 

th_cor = 0.5
maxx = np.zeros((1,np.array(rowp).shape[1]))
zeross = 0 #this is the total number of peaks that will set to zero
#for k in range (np.array(rowp).shape[1]): 
for k in range(np.array(rowp).shape[1]): #the peak counter    
    #print rowp[0][k] 
    #print(k)

    if all ([channels[k] != [] and np.array(channels[k]).shape[0] != 1]) : 
        #repetitive elements of rows since they have different rowp (peak time)
        #print(channels[k])
        maxx[0,k] = -999999
        #print(k)
        for p in range (np.array(channels[k])[:, None].shape[0]) : #considering number of same peaks in every group           
            #print p
            for t in range (np.array(channels[k])[:, None].shape[0]):
#                 print(t)
#                 print(p)

#below condition checks that channels be near each other (less than 3)             
                if all ([channels[k][p] != channels[k][t] and np.abs(channels[k][p]-channels[k][t]) < 3 and np.any(signals_after_corr[channels[k][p]][times[k][p]]) != 0 and np.any(signals_after_corr[channels[k][t]][times[k][t]]) != 0]):                 
                    #print(channels[k][p])
                    #print(channels[k][t])
                    corr = np.corrcoef(signals_after_corr[channels[k][p]][times[k][p]], signals_after_corr[channels[k][t]][times[k][t]])[0, 1]
                    #corr = np.correlate(cut_ch[channels[k][p]][k], cut_ch[channels[k][t]][k])
                    #plt.plot(signals_after_corr[channels[k][p]][times[k][p]],'r')
                    #plt.plot(signals_after_corr[channels[k][t]][times[k][t]])
                    plt.show()
                    print(corr)
                    if corr >= th_cor:  #if correlation is more than the threshold
#                         print('rowp=', channels[k][p])
#                         print('rowt=', channels[k][t])
                        if np.abs(X[channels[k][p],rowp[0, k]]) >= maxx[0, k]:
                            maxx[0, k] = np.abs(X[channels[k][p],rowp[0, k]])
                            #print(maxx[0,k])
                        else:    
                            signals_after_corr[channels[k][p]][times[k][p]] = np.zeros((1, T1)) #if that signal is not the max for correlation remove it 
                            zeross = zeross + 1
                            #print('rowp', channels[k][p]) #number of channel which is removed
                        if np.abs(X[channels[k][t],rowp[0, k]]) > maxx[0, k]: #X[...] is the amount of cut_ch (spike) in peak k
                            maxx[0, k] = np.abs(X[channels[k][t],rowp[0, k]])
                            signals_after_corr[channels[k][p]][times[k][p]] = np.zeros((1, T1))
                            #print('rowp', channels[k][p])
                            #print(maxx[0,k])
                            zeross = zeross + 1
                        else:    
                            signals_after_corr[channels[k][t]][times[k][t]] = np.zeros((1, T1))
                            zeross = zeross + 1
                        #print(maxx[0,k])    
print("--- %s seconds ---" % (time.time() - start_time))
                                

In [ ]:
# calculating total number of spikes
t = 0

for i in range (16):
    for k in range (np.array(signals_after_corr[i]).shape[0]): #for every peak
        if np.any(np.array(signals_after_corr[i])[k]) != 0:
            plt.plot(np.array(signals_after_corr[i])[k])
            t = t+1
    print(i+1)
    plt.show()    

In [ ]:
colors =['dodgerblue', 'skyblue', 'gray','deeppink','orange','saddlebrown','lawngreen', 'g','black','cyan','red','brown','lightgray','yellow','pink','magenta']
col = colors[7]

#plt.plot(np.array(cluster[3])[:, 0],np.array(cluster[3])[:, 1], col) 

In [ ]:
for i in range(np.array(signals_after_corr).shape[0]):
    final_spikes = []
    for j in range(np.array(signals_after_corr[i]).shape[0]):
        if np.all(signals_after_corr[i][j] == 0): 
            True_sig = 1  #print(cut_ch[i][j])
        else:
            final_spikes.append(signals_after_corr[i][j])
    if final_spikes != []:        
        print(i + 1) #this would be the channel number
        res = np.array(final_spikes)       
        varcovmat = np.cov(res.T)
        k = np.array(np.mean(res.T,1))
        tt = []
        for s in range(res.shape[0]):
            tt.append(k)
        varcovmat = res.T - np.array(tt).T

        u, s, v = svd(varcovmat)
        pca_data = np.dot(res,u[:,0:2])
        pca_data.shape

#         plt.scatter(pca_data[:,0],pca_data[:,1],c=colors[i])

#         #plt.xlim([-400,400])
#         #plt.ylim([-300,300])
#         plt.show()


In [ ]:
#Getting PCA of the data

num = t #this is calculated in the previous cell, giving the number of non-zero signals    
res = np.zeros((num, T1)) #put non-zero elements in res as arrays
chnum = np.zeros((num, 1)) #contains the channel number label for colors in plotting 

#for determining the labels and groups of clusters

clus_spike = []
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])
clus_spike.append([])

p = 0
for i in range (16):
    for j in range (np.array(signals_after_corr[i]).shape[0]):
        if (np.any(np.array(signals_after_corr[i])[j]) != 0): #measures the number of non zero elements (3d axis)
#             while p < np.array(signals_after_corr[i])[j].shape[0]:
            res[p,:] = np.array(signals_after_corr[i])[j]
            chnum[p,0] = i
            data_ch = [i] #these are the 64 signals 
            p = p + 1
            if i == 0:
                clus_spike[0].append(np.array(signals_after_corr[i])[j])
            if i == 1:
                clus_spike[1].append(np.array(signals_after_corr[i])[j])
            if i == 2:
                clus_spike[2].append(np.array(signals_after_corr[i])[j])
            if i == 3:
                clus_spike[3].append(np.array(signals_after_corr[i])[j])
            if i == 4:
                clus_spike[4].append(np.array(signals_after_corr[i])[j])
            if i == 5:
                clus_spike[5].append(np.array(signals_after_corr[i])[j])
            if i == 6:
                clus_spike[6].append(np.array(signals_after_corr[i])[j])
            if i == 7:
                clus_spike[7].append(np.array(signals_after_corr[i])[j]) 
            if i == 8:
                clus_spike[8].append(np.array(signals_after_corr[i])[j])
            if i == 9:
                clus_spike[9].append(np.array(signals_after_corr[i])[j])
            if i == 10:
                clus_spike[10].append(np.array(signals_after_corr[i])[j])
            if i == 11:
                clus_spike[11].append(np.array(signals_after_corr[i])[j])
            if i == 12:
                clus_spike[12].append(np.array(signals_after_corr[i])[j])
            if i == 13:
                clus_spike[13].append(np.array(signals_after_corr[i])[j])
            if i == 14:
                clus_spike[14].append(np.array(signals_after_corr[i])[j])
            if i == 15:
                clus_spike[15].append(np.array(signals_after_corr[i])[j])
          
            
#np.save('clus_spike', clus_spike)            

varcovmat = np.cov(res.T)
k = np.array(np.mean(res.T,1))
tt = []
for i in range(res.shape[0]):
   tt.append(k)
varcovmat = res.T - np.array(tt).T

u, s, v = svd(varcovmat)
pca_data = np.dot(res,u[:,0:2])
pca_data.shape

for i in range(chnum.shape[0]):    
    plt.scatter(pca_data[i,0],pca_data[i,1], c=colors[int(chnum[i,0])])
                
#plt.xlim([-400,400])
#plt.ylim([-300,300])
plt.show()



In [ ]:
# Use GEMsort to cluster the data

%matplotlib qt 

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
T_cluster = 50 # threshold for the number of spikes to be considered as a cluster

ch_num = 16
for s in range(ch_num):
#     print('s=',s)
    ch_pca = []

    for i in range(pca_data.shape[0]): 
        #print(int(chnum[i,0]))
        if s == int(chnum[i,0]):
            ch_pca.append(list(pca_data[i,:]))
    ch_pca = np.array(ch_pca)  
    if ch_pca.shape[0] > T_cluster: #if there is data
        ax.plot(ch_pca[:,0], ch_pca[:,1], s+1, linestyle="none", marker="o", markersize=3, color=colors[int(s)],zorder=1)#c=colors[int(s)],zorder=1)     
        ax.set_zlim(np.array([0,32])*0.5)
    if ch_pca.shape[0] > T_cluster:
        prenumnode = 7 
        GEMsort_dataset = ch_pca
        Nodes,GEMsort_C,GEMsort_w, nodes_total,prenumnode,d,nodess = GEMsort(prenumnode, GEMsort_dataset)
        
        for i in range(GEMsort_C.shape[0]):
            for j in range (GEMsort_C.shape[0]):
                if GEMsort_C[i,j] != -1:
                   # print i,j
                    x = [GEMsort_w[i,0],GEMsort_w[j,0]]
                    y = [GEMsort_w[i, 1], GEMsort_w[j, 1]]
                    ax.scatter(x, y, s+1, s=100, c ='r', zorder=3)
                    ax.plot(x, y, s+1,'k', zorder=2, lw=2) 

ax.set_xlabel('PC1', fontsize=15, rotation=0)
ax.set_ylabel('PC2', fontsize=15)
ax.set_zlabel('Channel Number', fontsize=15, rotation=60)

ax.set_zticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
#ax.set_xlim(left, right)
#ax.set_ylim(-0.1, 0.4)

plt.show()
